In [1]:
import torch
import torchvision
import importlib

!pip install --upgrade torchvision
!pip install cython
!pip install -U "git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI"

Requirement already up-to-date: torchvision in /Users/kevinmiao/opt/anaconda3/lib/python3.7/site-packages (0.9.1)
  Cloning https://github.com/cocodataset/cocoapi.git to /private/var/folders/1f/d9_sk_gj4_l9r3my4qtylk5r0000gn/T/pip-req-build-1oebq7eh
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /private/var/folders/1f/d9_sk_gj4_l9r3my4qtylk5r0000gn/T/pip-req-build-1oebq7eh
  ERROR: Command errored out with exit status 1:
   command: /Users/kevinmiao/opt/anaconda3/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/1f/d9_sk_gj4_l9r3my4qtylk5r0000gn/T/pip-req-build-1oebq7eh/PythonAPI/setup.py'"'"'; __file__='"'"'/private/var/folders/1f/d9_sk_gj4_l9r3my4qtylk5r0000gn/T/pip-req-build-1oebq7eh/PythonAPI/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/1f/d9_sk_gj4_

Failed to build pycocotools
  Running setup.py install for pycocotools ... error
    ERROR: Command errored out with exit status 1:
     command: /Users/kevinmiao/opt/anaconda3/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/1f/d9_sk_gj4_l9r3my4qtylk5r0000gn/T/pip-req-build-1oebq7eh/PythonAPI/setup.py'"'"'; __file__='"'"'/private/var/folders/1f/d9_sk_gj4_l9r3my4qtylk5r0000gn/T/pip-req-build-1oebq7eh/PythonAPI/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /private/var/folders/1f/d9_sk_gj4_l9r3my4qtylk5r0000gn/T/pip-record-9du8gdhv/install-record.txt --single-version-externally-managed --compile
         cwd: /private/var/folders/1f/d9_sk_gj4_l9r3my4qtylk5r0000gn/T/pip-req-build-1oebq7eh/PythonAPI
    Complete output (26 lines):
    running install
    running build
    running build_py
    running bu

In [2]:
torch.__version__

'1.8.1'

In [12]:
from dataset import *

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
### HYPERPARAM

batchsize = 1
in_dim = (300,300)
normalization_data = torch.load('mean-std.pt')
num_classes = 5

In [5]:
import wandb
wandb.init(project="FSDL-Skin")

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: kevin-miao (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.27 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [6]:
# !cd Desktop/FSDL-FinalProject/dataverse_files/HAM10000_images_part_1/
# !ls
# from PIL import Image
# Image.open('ISIC_0024306.jpg')

In [7]:
# precision = 'fp32'
# pretrained=False
# ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd',
#                            model_math=precision, pretrained=False)

# if precision == 'fp16':
#     checkpoint_url = 'https://api.ngc.nvidia.com/v2/models/nvidia/ssdpyt_fp16/versions/1/files/nvidia_ssdpyt_fp16_20190225.pt'
# elif precision == 'fp32':
#     checkpoint_url = 'https://api.ngc.nvidia.com/v2/models/nvidia/ssdpyt_fp32/versions/1/files/nvidia_ssdpyt_fp32_20190225.pt'

# checkpoint = torch.hub.load_state_dict_from_url('https://api.ngc.nvidia.com/v2/models/nvidia/ssdpyt_fp32/versions/1/files/nvidia_ssdpyt_fp32_20190225.pt', map_location="cpu")
# state_dict = {key.replace("module.", ""): value for key, value in checkpoint["state_dict"].items()}
# ssd_model.load_state_dict(state_dict)

In [10]:
import torchvision.transforms as transforms

In [13]:
%load_ext autoreload
%autoreload 2
import dataset
dataset = dataset.SkinData('/', 'final.csv', transform=transforms.Compose([Resizing(in_dim), ToTensor, Normalizer(normalization_data)]))
train_data, test_data = torch.utils.data.random_split(dataset,[int(0.8 * len(dataset)), int(0.2 * len(dataset))],  generator=torch.Generator().manual_seed(42))
data_loader_train = torch.utils.data.DataLoader(train_data, batch_size=batchsize)
data_loader_test = torch.utils.data.DataLoader(test_data, batch_size=batchsize)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)


    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [15]:
 model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)


In [107]:
target['boxes'].shape

torch.Size([4])

In [23]:
target

{'area': tensor([19819.3359]),
 'labels': tensor([0, 0, 0, 0, 0, 1, 0]),
 'boxes': tensor([ 79.5000,  94.6667, 216.5000, 239.3333]),
 'image_id': tensor([0])}

In [24]:
model(im, [target])

ValueError: Expected target boxes to be a tensorof shape [N, 4], got torch.Size([4]).

In [83]:
# Set device type
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Load pre-trained model with new head/tail
model = get_instance_segmentation_model(num_classes)
model.to(device)

# Build optimizer
params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.005,
#                             momentum=0.9, weight_decay=0.0005)
optimizer = torch.optim.AdamW(params, lr=0.005, weight_decay=0.0005)

# Learning rate scheduler (LR decay)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [17]:
image, target = dataset[0]

In [18]:
image.shape

torch.Size([3, 300, 300])

In [19]:
model(image, target)

TypeError: string indices must be integers

In [21]:
im = image.unsqueeze(0)
im.shape

torch.Size([1, 3, 300, 300])

In [74]:
with torch.no_grad():
    model.eval()
    out = model(im)
out

[{'boxes': tensor([[135.5351, 161.7220, 136.9277, 168.7584],
          [113.6921, 145.8676, 245.1281, 164.7671],
          [253.3619,   0.0000, 294.8402, 180.3698],
          [235.2942,   0.0000, 289.5670, 276.4473],
          [150.2459, 240.4107, 255.4346, 274.9097],
          [ 28.7036,   0.0000,  76.4916, 214.9891],
          [212.6953, 229.9816, 278.8131, 274.2871],
          [ 38.4179,   0.0000, 104.7946, 203.0995],
          [  0.0000, 239.4225,  56.7588, 273.8826],
          [  0.0000, 182.3397,  34.5776, 248.0956],
          [  0.0000, 206.4940,  45.4838, 259.2822],
          [175.0613, 258.7690, 267.4514, 284.7316],
          [ 14.7753,   0.0000,  41.4793, 268.5659],
          [ 52.5961,   0.0000, 143.3818, 179.2598],
          [157.6584, 102.5488, 169.7441, 109.4422],
          [110.2645, 223.6642, 117.7944, 240.5923],
          [129.2219, 237.6158, 265.8083, 300.0000],
          [248.0948,  99.4939, 263.1408, 108.6464],
          [182.6748,  88.1443, 185.4039,  91.1719],
   

In [48]:
from engine import train_one_epoch, evaluate

# Training Loop
num_epochs = 10

for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=10)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)

AttributeError: 'str' object has no attribute 'items'